# Uplift Modeling with EconML using MovieLens 1M
This notebook downloads MovieLens 1M data, simulates treatment and renewal outcomes, introduces missing data, imputes values, and trains S-, T-, and X-Learners using different base learners.

In [ ]:
!pip uninstall  econml scikit-learn pandas numpy

Found existing installation: econml 0.15.1
Uninstalling econml-0.15.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/econml-0.15.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/econml/*
Proceed (Y/n)? 

In [ ]:
!pip  install --no-cache-dir  econml scikit-learn pandas numpy

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from econml.metalearners import SLearner, TLearner, XLearner

In [151]:
# Download and extract MovieLens 1M dataset
!pip install wget
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

--2025-05-18 02:51:11--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.3’

ml-1m.zip.3         100%[===================>]   5.64M  5.87MB/s    in 1.0s    

2025-05-18 02:51:12 (5.87 MB/s) - ‘ml-1m.zip.3’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   


In [152]:
# Download and extract MovieLens 1M dataset
# The wget and unzip commands appear to be working correctly based on your output.
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

# Add checks to verify if the directory and file exist
import os

# Correct the path to reflect the nested directory structure
if os.path.exists('ml-1m/ml-1m/ratings.dat'):
    print("ml-1m/ml-1m/ratings.dat found. Proceeding to load data.")
else:
    print("Error: ml-1m/ml-1m/ratings.dat not found. Please check the extraction path.")
    # If the file is still not found after correcting the path, there might be
    # a deeper issue with the unzip process or disk.
    # import sys
    # sys.exit(1) # Uncomment to exit the notebook execution if the file is not found

--2025-05-18 02:51:14--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.4’

ml-1m.zip.4         100%[===================>]   5.64M  5.94MB/s    in 1.0s    

2025-05-18 02:51:16 (5.94 MB/s) - ‘ml-1m.zip.4’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   
ml-1m/ml-1m/ratings.dat found. Proceeding to load data.


In [153]:
# Load data
ratings = pd.read_csv('ml-1m//ml-1m/ratings.dat', sep='::', engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
users = pd.read_csv('ml-1m/ml-1m/users.dat', sep='::', engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
#movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
#                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
#df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [ ]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings.shape

(1000209, 4)

In [ ]:
ratings.isnull().values.sum()

0

In [ ]:
ratings.isnull().sum()

,0
UserID,0
MovieID,0
Rating,0
Timestamp,0


In [ ]:
ratings.isnull().any()

,0
UserID,False
MovieID,False
Rating,False
Timestamp,False


In [ ]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   UserID      6040 non-null   int64 
 1   Gender      6040 non-null   object
 2   Age         6040 non-null   int64 
 3   Occupation  6040 non-null   int64 
 4   Zip-code    6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [ ]:
users.isnull().values.sum()

0

In [ ]:
users.value_counts()

,,,,,count
UserID,Gender,Age,Occupation,Zip-code,
1,F,1,10,48067,1
4024,M,25,5,45011,1
4033,M,25,6,43551,1
4032,M,25,12,56301,1
4031,M,25,0,42445,1
...,...,...,...,...,...
2012,M,25,4,49456,1
2011,F,35,3,01545,1
2010,M,18,4,81520,1


In [ ]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings.groupby('UserID')

In [ ]:
ratings.groupby('UserID').get_group(1)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [ ]:
ratings.groupby('UserID')['MovieID'].count().rename('MovieID_Count').reset_index().sort_values(by = 'MovieID_Count', ascending =False)

,UserID,MovieID_Count
4168,4169,2314
1679,1680,1850
4276,4277,1743
1940,1941,1595
1180,1181,1521
...,...,...
5724,5725,20
3406,3407,20
1663,1664,20
4418,4419,20


In [ ]:
ratings.groupby('UserID')[['MovieID', 'Rating']].count().rename(columns = {'MovieID':'MovieID_Count','Rating':'Rating_Count' }).reset_index().sort_values(by = 'MovieID_Count', ascending =False)

,UserID,MovieID_Count,Rating_Count
4168,4169,2314,2314
1679,1680,1850,1850
4276,4277,1743,1743
1940,1941,1595,1595
1180,1181,1521,1521
...,...,...,...
5724,5725,20,20
3406,3407,20,20
1663,1664,20,20
4418,4419,20,20


In [ ]:
ratings.groupby('UserID').agg({'MovieID':'count', 'Rating':'nunique'}).reset_index().rename(columns = {'MovieID':"MovieID_count", 'Rating':'Unique_Rating_count'})

,UserID,MovieID_count,Unique_Rating_count
0,1,53,3
1,2,129,5
2,3,51,5
3,4,21,5
4,5,198,5
...,...,...,...
6035,6036,888,5
6036,6037,202,5
6037,6038,20,5
6038,6039,123,4


In [ ]:
ratings['UserID'].nunique()

6040

In [ ]:
ratings.nunique()

,0
UserID,6040
MovieID,3706
Rating,5
Timestamp,458455


In [ ]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
ratings.head()


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [154]:
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
#df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [ ]:
movies.head(1)

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy


In [ ]:
users.head(1)

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067


In [ ]:
ratings.head(1)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760


In [155]:
df = ratings.merge(users, on = 'UserID').merge(movies, on = 'MovieID')
df.sample(10)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
552083,3401,3066,3,980113845,M,35,7,76109,Tora! Tora! Tora! (1970),War
59785,406,605,4,976655494,M,25,20,55105,One Fine Day (1996),Drama|Romance
547078,3371,1242,2,967610026,M,25,2,55405,Glory (1989),Action|Drama|War
753609,4489,2048,3,1035297376,M,18,4,78613,"Great Mouse Detective, The (1986)",Animation|Children's
155684,1004,743,3,975053069,M,25,3,95136,Spy Hard (1996),Comedy
456592,2810,2763,3,972794576,F,1,10,64093,"Thomas Crown Affair, The (1999)",Action|Thriller
128982,838,2527,3,975360387,M,50,17,92661,Westworld (1973),Action|Sci-Fi|Thriller|Western
781051,4663,1480,1,963867530,M,25,4,92037,Smilla's Sense of Snow (1997),Action|Drama|Thriller
528783,3269,1265,5,968198784,F,18,20,45220,Groundhog Day (1993),Comedy|Romance
445550,2748,1894,1,973209367,M,25,4,85719,Six Days Seven Nights (1998),Adventure|Comedy|Romance


In [ ]:
users['UserID'].nunique()

6040

In [ ]:
ratings['UserID'].nunique()

6040

In [ ]:
df['UserID'].nunique()

6040

In [ ]:
df.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'Gender', 'Age',
       'Occupation', 'Zip-code', 'Title', 'Genres'],
      dtype='object')

In [ ]:
df.dtypes

,0
UserID,int64
MovieID,int64
Rating,int64
Timestamp,int64
Gender,object
Age,int64
Occupation,int64
Zip-code,object
Title,object
Genres,object


In [ ]:
df.Genres.nunique()

301

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   UserID      1000209 non-null  int64 
 1   MovieID     1000209 non-null  int64 
 2   Rating      1000209 non-null  int64 
 3   Timestamp   1000209 non-null  int64 
 4   Gender      1000209 non-null  object
 5   Age         1000209 non-null  int64 
 6   Occupation  1000209 non-null  int64 
 7   Zip-code    1000209 non-null  object
 8   Title       1000209 non-null  object
 9   Genres      1000209 non-null  object
dtypes: int64(6), object(4)
memory usage: 76.3+ MB


In [ ]:
df.dtypes

,0
UserID,int64
MovieID,int64
Rating,int64
Timestamp,int64
Gender,object
Age,int64
Occupation,int64
Zip-code,object
Title,object
Genres,object


In [ ]:
df['Genres'] =df['Genres'].astype('category')
df['Gender'] =df['Gender'].astype('category')
df['Rating'] =df['Rating'].astype('category')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype   
---  ------      --------------    -----   
 0   UserID      1000209 non-null  int64   
 1   MovieID     1000209 non-null  int64   
 2   Rating      1000209 non-null  category
 3   Timestamp   1000209 non-null  int64   
 4   Gender      1000209 non-null  category
 5   Age         1000209 non-null  int64   
 6   Occupation  1000209 non-null  int64   
 7   Zip-code    1000209 non-null  object  
 8   Title       1000209 non-null  object  
 9   Genres      1000209 non-null  category
dtypes: category(3), int64(5), object(2)
memory usage: 57.2+ MB


In [ ]:
df.nunique()

,0
UserID,6040
MovieID,3706
Rating,5
Timestamp,458455
Gender,2
Age,7
Occupation,21
Zip-code,3439
Title,3706
Genres,301


In [ ]:
df.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [ ]:
df['Age'].value_counts()

,count
Age,
25,395556
35,199003
18,183536
45,83633
50,72490
56,38780
1,27211


In [ ]:
df.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
364045,2121,3,3,974646826,M,56,13,02356,Grumpier Old Men (1995),Comedy|Romance
599503,3648,377,3,994800533,M,18,4,85210,Speed (1994),Action|Romance|Thriller
961878,5795,1747,5,958147550,M,25,1,92688,Wag the Dog (1997),Comedy|Drama
890957,5380,3142,3,960392672,M,18,4,01125,U2: Rattle and Hum (1988),Documentary|Musical
515745,3182,2795,5,968774819,M,25,12,12866,Vacation (1983),Comedy


In [ ]:
df.shape, len(df)

((1000209, 10), 1000209)

In [156]:
np.random.seed(88)
np.random.uniform(10,20,len(df))

array([16.47551049, 15.07149688, 15.2834138 , ..., 14.7004425 ,
       19.29304266, 18.10024389])

In [157]:
# Feature creation
np.random.seed(42)
df['WatchTime'] = df['Rating'] * np.random.uniform(15, 30, size=len(df)).astype(int)


In [158]:
df.sample(2)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime
954034,5759,1732,5,959559219,F,25,1,08904,"Big Lebowski, The (1998)",Comedy|Crime|Mystery|Thriller,145
184522,1147,1238,3,974873713,M,25,20,98101,Local Hero (1983),Comedy,60


In [ ]:
df['Timestamp'].max(), df['Timestamp'].min()

(1046454590, 956703932)

In [ ]:
pd.to_datetime(df['Timestamp'], unit='s')

,Timestamp
0,2000-12-31 22:12:40
1,2000-12-31 22:35:09
2,2000-12-31 22:32:48
3,2000-12-31 22:04:35
4,2001-01-06 23:38:11
...,...
1000204,2000-04-26 02:35:41
1000205,2000-04-25 23:21:27
1000206,2000-04-25 23:19:06
1000207,2000-04-26 02:20:48


In [159]:
df['Timestamp_Date'] = pd.to_datetime(df['Timestamp'], unit='s').dt.strftime('%Y-%m-%d')

In [160]:
df.sample(3)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date
734620,4387,3672,2,965433984,F,18,4,63109,Benji (1974),Adventure|Children's,30,2000-08-05
714252,4277,3753,5,978077207,M,35,16,98133,"Patriot, The (2000)",Action|Drama|War,80,2000-12-29
240635,1449,2952,3,975605335,M,35,20,17112,"Hard 8 (a.k.a. Sydney, a.k.a. Hard Eight) (1996)",Crime|Thriller,63,2000-11-30


In [ ]:
df['Timestamp_Date'].max(), df['Timestamp_Date'].min()

('2003-02-28', '2000-04-25')

In [166]:
df['TenureMonths'] = (df['Timestamp'] - df['Timestamp'].min()) // (60*60*24*30)

In [ ]:
df.sample(3)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths
535803,3308,3247,4,967977490,F,18,20,15701-1348,Sister Act (1992),Comedy|Crime,112,2000-09-03,4
405577,2429,508,4,975258915,M,25,2,22903,Philadelphia (1993),Drama,72,2000-11-26,7
554198,3410,36,4,967471944,M,35,1,20653,Dead Man Walking (1995),Drama,68,2000-08-28,4


In [167]:
df.head(5)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,100,2000-12-31,8
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,87,2000-12-31,8
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance,75,2000-12-31,8
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama,92,2000-12-31,8
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,85,2001-01-06,8


In [168]:
df2 = pd.DataFrame()
df2['UserID'] = df['UserID'].drop_duplicates()
df2.head()

,UserID
0,1
53,2
182,3
233,4
254,5


In [169]:
df2['Age'] = np.random.randint(18, 70, df2.shape[0])
df2.head()

,UserID,Age
0,1,67
53,2,60
182,3,44
233,4,66
254,5,55


In [ ]:
df.head(2)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,100,2000-12-31,8
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,87,2000-12-31,8


In [170]:
df_user = df.merge(df2, on = 'UserID', how = 'left')
df_user.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Age_x,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age_y
782374,4673,3100,5,964489733,M,35,16,94705,"River Runs Through It, A (1992)",Drama,100,2000-07-25,3,56
441311,2702,551,3,973300850,M,45,6,65201,"Nightmare Before Christmas, The (1993)",Children's|Comedy|Musical,87,2000-11-04,6,54
289354,1727,1513,4,974706110,M,35,16,50317,Romy and Michele's High School Reunion (1997),Comedy,108,2000-11-20,6,61
531522,3281,507,4,968168634,M,25,17,79705,"Perfect World, A (1993)",Action|Drama,88,2000-09-05,4,64
401588,2395,2683,4,1041266615,M,35,7,08021,Austin Powers: The Spy Who Shagged Me (1999),Comedy,92,2002-12-30,32,42


In [171]:
df_user = df_user.drop('Age_x', axis= 1).rename(columns = {'Age_y':'Age'})
df_user.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age
223986,1356,737,1,974783916,F,4,91706,Barb Wire (1996),Action|Sci-Fi,17,2000-11-21,6,50
505556,3110,3418,3,969487772,M,0,10002,Thelma & Louise (1991),Action|Drama,60,2000-09-20,4,41
827396,4969,1927,2,962606637,M,19,94111,All Quiet on the Western Front (1930),War,30,2000-07-03,2,59
982306,5927,2626,5,957283526,M,14,10003,Edge of Seventeen (1998),Comedy|Drama|Romance,95,2000-05-02,0,60
131831,854,1875,3,975352114,F,16,44092,Clockwatchers (1997),Comedy,75,2000-11-27,7,40


In [ ]:
df_user.shape

(1000209, 13)

In [ ]:
df_tmp = pd.DataFrame({
    'Category': ['A', 'A', 'B', 'B', 'C'],
    'Value': [10, 20, 30, 40, 50],
    'Other': ['foo', 'bar', 'baz', 'qux', 'quux']
})
df_tmp


,Category,Value,Other
0,A,10,foo
1,A,20,bar
2,B,30,baz
3,B,40,qux
4,C,50,quux


In [ ]:
df_tmp.groupby('Category').first()

,Value,Other
Category,,
A,10,foo
B,30,baz
C,50,quux


In [172]:
# Feature creation
user_features = df_user.groupby('UserID').agg({
    'WatchTime': 'sum',
    'MovieID': 'nunique',
    'TenureMonths': 'max',
    'Age': 'first',
    'Occupation': 'first'
}).rename(columns={'WatchTime': 'TotalWatchTime', 'MovieID': 'UniqueMovies'})

In [ ]:
user_features.sample(5)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
1533,6625,75,6,45,1
5682,50407,608,34,18,0
423,3630,64,7,18,4
4118,18559,207,10,25,3
486,3291,39,7,56,0


In [ ]:
df_user

,UserID,MovieID,Rating,Timestamp,Gender,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age
0,1,1193,5,978300760,F,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,100,2000-12-31,8,59
1,1,661,3,978302109,F,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,87,2000-12-31,8,59
2,1,914,3,978301968,F,10,48067,My Fair Lady (1964),Musical|Romance,75,2000-12-31,8,59
3,1,3408,4,978300275,F,10,48067,Erin Brockovich (2000),Drama,92,2000-12-31,8,59
4,1,2355,5,978824291,F,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,85,2001-01-06,8,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,6,11106,Weekend at Bernie's (1989),Comedy,28,2000-04-26,0,19
1000205,6040,1094,5,956704887,M,6,11106,"Crying Game, The (1992)",Drama|Romance|War,90,2000-04-25,0,19
1000206,6040,562,5,956704746,M,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,130,2000-04-25,0,19
1000207,6040,1096,4,956715648,M,6,11106,Sophie's Choice (1982),Drama,96,2000-04-26,0,19


In [ ]:
user_features.sample(4)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
2409,11263,134,20,35,9
4038,2073,23,3,50,6
2561,6531,77,8,25,15
5168,2005,20,2,35,7


In [ ]:
user_features.sample(frac=0.001).index

Index([2493, 5638, 4262, 1639, 880, 4403], dtype='int64', name='UserID')

In [173]:
# Introduce and impute missing data
user_features.loc[user_features.sample(frac=0.1).index, 'TotalWatchTime'] = np.nan
user_features.sample(10)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
2731,26099.0,280,6,39,7
3720,15957.0,207,3,19,7
5820,3735.0,48,0,34,0
4130,13999.0,172,3,59,6
1853,14205.0,157,7,69,0
146,36262.0,426,33,42,20
421,NaN,20,7,67,3
3419,10440.0,120,4,58,1
1424,3300.0,44,6,57,0


In [ ]:
user_features['TotalWatchTime'].isnull().sum()

604

In [174]:
user_features.loc[user_features.sample(frac=0.1).index, 'TenureMonths'] = np.nan
user_features.sample(15)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
3677,12151.0,141,3.0,41,12
2535,5721.0,66,15.0,42,2
4805,3068.0,35,15.0,65,1
5931,6016.0,77,NaN,58,7
5078,NaN,60,2.0,62,7
2969,13882.0,174,32.0,60,7
1650,17214.0,213,11.0,50,19
777,28339.0,502,NaN,45,19
834,4538.0,50,7.0,51,12


In [175]:
# Introduce and impute missing data
user_features['TotalWatchTime'] =user_features['TotalWatchTime'].fillna(user_features['TotalWatchTime'].median())
user_features['TenureMonths']= user_features['TenureMonths'].fillna(user_features['TenureMonths'].median())

In [ ]:
user_features.isnull().sum()

,0
TotalWatchTime,0
UniqueMovies,0
TenureMonths,0
Age,0
Occupation,0


In [176]:
# Simulate treatment and renewal
user_features['treatment'] = np.random.binomial(1, 0.5, size=len(user_features))
engaged = user_features['TotalWatchTime'] > user_features['TotalWatchTime'].median()
base_rate = 0.2
uplift = 0.15 * ((user_features['treatment'] == 1) & engaged).astype(float)
user_features['renewed'] = np.random.binomial(1, base_rate + uplift)
X = user_features[['TenureMonths', 'TotalWatchTime', 'UniqueMovies']]
T = user_features['treatment'].values
Y = user_features['renewed'].values

In [177]:
# Split data
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X, T, Y, test_size=0.2, random_state=42)

In [195]:
X_train.head(10)

,TenureMonths,TotalWatchTime,UniqueMovies
UserID,,,
1122,7.0,8331.0,114
4432,21.0,6121.0,68
4061,3.0,2799.0,38
810,7.0,10727.0,126
4582,33.0,4334.0,46
2580,6.0,7761.5,74
5948,6.0,37033.0,398
5210,3.0,7761.5,65
4251,3.0,7510.0,97


In [198]:
X_train.columns.to_list()

['TenureMonths', 'TotalWatchTime', 'UniqueMovies']

In [202]:
X_train.select_dtypes(include=['number'])

,TenureMonths,TotalWatchTime,UniqueMovies
UserID,,,
1122,7.0,8331.0,114
4432,21.0,6121.0,68
4061,3.0,2799.0,38
810,7.0,10727.0,126
4582,33.0,4334.0,46
...,...,...,...
3773,30.0,7761.5,361
5192,1.0,1654.0,22
5227,1.0,10723.0,157


In [201]:
X_train.select_dtypes(include=['number']).columns

Index(['TenureMonths', 'TotalWatchTime', 'UniqueMovies'], dtype='object')

In [ ]:
#feature scaling
# # Preprocessing
# numeric_features = ["tenure_months", "prior_engagement_score", "weekly_watch_hours", "num_devices"]
# categorical_features = ["device_type", "payment_method", "account_type", "region", "has_kids_profile", "promo_eligible"]

# preprocessor = ColumnTransformer([
#     ("num", StandardScaler(), numeric_features),
#     ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
# ])

# # Fit and transform
# X_train_proc = preprocessor.fit_transform(X_train)
# X_test_proc = preprocessor.transform(X_test)
# # 🎯 Evaluate both
# print("Sklearn GBM:")
# print(classification_report(y_test, sk_gbm.predict(X_test_proc)))
# print("AUC:", roc_auc_score(y_test, sk_gbm.predict_proba(X_test_proc)[:, 1]))


In [206]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score

In [209]:
numeric_features  = X_train.select_dtypes(include=['number']).columns.tolist()
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features)
 #   ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])

# Fit and transform
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

In [214]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': [ 'liblinear'],
    'class_weight': [None, 'balanced'],
    'max_iter': [100, 200,500,1000]
    }



lr_grid = GridSearchCV(LogisticRegression(), param_grid, cv=3)
lr_grid.fit(X_train_proc, Y_train)
lr_grid.best_score_, lr_grid.best_params_
best_lr = lr_grid.best_estimator_

In [215]:

lr_grid.best_score_, lr_grid.best_params_

(0.7628307199596973,
 {'C': 10,
  'class_weight': None,
  'max_iter': 100,
  'penalty': 'l2',
  'solver': 'liblinear'})

In [218]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': [ 'saga'],
    'class_weight': [None, 'balanced'],
    'max_iter': [10000, 20000,50000],
     'l1_ratio': [0, 0.25, 0.5, 0.75, 1]
}

lr_grid = GridSearchCV(LogisticRegression(), param_grid, cv=3)
lr_grid.fit(X_train_proc, Y_train)
lr_grid.best_score_, lr_grid.best_params_
best_lr = lr_grid.best_estimator_

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
l1_ratio parameter is only used when penalty is 'ela

In [219]:
lr_grid.best_score_, lr_grid.best_params_

(0.7628307199596973,
 {'C': 10,
  'class_weight': None,
  'l1_ratio': 0,
  'max_iter': 10000,
  'penalty': 'l2',
  'solver': 'saga'})

In [210]:
lr_grid.best_score_

0.7622102445274658

In [189]:
lr_grid.best_score_

0.7628307199596973

In [186]:
lr_grid.best_params_

{'C': 10,
 'class_weight': None,
 'max_iter': 200,
 'penalty': 'l1',
 'solver': 'liblinear'}

In [5]:
# Hyperparameter tuning
lr_grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid={'C': [0.01, 0.1, 1, 10]}, cv=3)
lr_grid.fit(X_train, Y_train)
best_lr = lr_grid.best_estimator_

rf_random = RandomizedSearchCV(RandomForestRegressor(random_state=42),
    param_distributions={'n_estimators': [100, 200], 'max_depth': [None, 10, 20]},
    n_iter=4, cv=3, random_state=42)
rf_random.fit(X_train, Y_train)
best_rf = rf_random.best_estimator_

gb_grid = GridSearchCV(GradientBoostingRegressor(random_state=42),
    param_grid={'n_estimators': [100, 150], 'learning_rate': [0.05, 0.1]}, cv=3)
gb_grid.fit(X_train, Y_train)
best_gb = gb_grid.best_estimator_

NameError: name 'GridSearchCV' is not defined

In [235]:
!pip uninstall econml

Found existing installation: econml 0.15.1
Uninstalling econml-0.15.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/econml-0.15.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/econml/*
Proceed (Y/n)? y
  Successfully uninstalled econml-0.15.1


In [236]:
!pip install econml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.3 MB/s eta 0:00:00


In [1]:
import econml

In [4]:
# Train learners
from econml.metalearners import SLearner, TLearner, XLearner # Re-import the learners
s_learner = SLearner(best_lr)
t_learner = TLearner(best_rf)
x_learner = XLearner(best_gb)
s_learner.fit(Y_train, T_train, X=X_train)
t_learner.fit(Y_train, T_train, X=X_train)
x_learner.fit(Y_train, T_train, X=X_train)
s_te = s_learner.effect(X_test)
t_te = t_learner.effect(X_test)
x_te = x_learner.effect(X_test)
pd.DataFrame({'S_Learner': s_te, 'T_Learner': t_te, 'X_Learner': x_te}).head()

NameError: name 'best_lr' is not defined

In [3]:
print(econml.__version__)

0.15.1


In [234]:
type(best_lr)

sklearn.linear_model._logistic.LogisticRegression